In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
MODEL = "mixtral:8x7b"
MODEL = "llama2"

In [2]:
import os
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

MODEL = "tinyllama"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "sk-your-openai-api-key"  

if MODEL.startswith("gpt"):
    if not OPENAI_API_KEY or OPENAI_API_KEY == "sk-your-openai-api-key":
        raise ValueError("❌ OPENAI_API_KEY is missing! Provide it as env variable or hardcode it.")
    
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

response = model.invoke("Tell me a joke!")
print(response)


Sure, here's a joke for you:

Q: What do you call an AI that can mow the lawn?
A: An AIYah!

This joke is about a computer program or software designed to perform tasks, like mowing lawns. It has a silly and entertaining name. Enjoy!


In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

'Sorry for the late response! Here\'s a classic AI joke:\n\nAI: "Hey, can you tell me a joke?"\nFriend: "Sure, what\'s the secret to a long and happy marriage?"\nAI: "Uh, the secret is having an open relationship."'

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "People call me Pareeshi", "question": "What's your name'?"})

'Answer: There is no need to "call" a person by their name. The only appropriate response to the question is "I don\'t know."'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("doc.pdf")
pages = loader.load_and_split()
pages

invalid pdf header: b'\xd9/\x00\x00%'
incorrect startxref pointer(1)
parsing for Object Streams


[Document(metadata={'source': 'doc.pdf', 'page': 0}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 1 of 3 \nPETITIONER:\nK.M. SHARMA\n        Vs.\nRESPONDENT:\nDEVI LAL & ORS.\nDATE OF JUDGMENT09/01/1990\nBENCH:\nMISRA RANGNATH\nBENCH:\nMISRA RANGNATH\nPUNCHHI, M.M.\nCITATION:\n 1990 AIR  528            1990 SCR  (1)   1\n 1990 SCC  (1) 438        JT 1990 (1)     5\n 1990 SCALE  (1)1\nACT:\n    Constitution of India, 1950: Article 75(4) and  Schedule\n111  Forms I and II--Oath/Affirmation by  Minister--Descrip-\ntion  of  person as Deputy  Prime  Minister--Whether  valid,\nlegal and constitutional.\nHEADNOTE:\n    The petitioner challenged the appointment of  Respondent\nNo.  1 as Deputy Prime Minister of India on the ground  that\nthe  oath  administered to him as such was not the  oath  in\naccordance with the prescription of the Constitution.\n    The  petition was contested by Respondent No. 1 and  the\nUnion  of  India. It was contended for the latter  that  de-\ns

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke("machine learning")

[Document(metadata={'source': 'doc.pdf', 'page': 2}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 3 of 3 \ncerned, the petitioner himself does not press it.\n    We accordingly reject the writ petition for the  reasons\nindicated above. No costs.\nP.S.   S                                            Petition\ndismissed.\n4'),
 Document(metadata={'source': 'doc.pdf', 'page': 1}, page_content='http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 2 of 3 \n    (Under Article 32 of the Constitution of India).\n    K.M. Sharma petitioner-in-person.\n    Soli  J.  Sorabjee,  Attorney  General,  Santosh  Hegde,\nAdditional Solicitor General, A.K. Sen, Manbir Singh, Girish\nChandra and Ms. A. Subhashini for the Respondents.\n   The Court delivered the following order:\n                      ORDER\n    Two  contentions were raised by the petitioner  in  this\nwrit petition challenging the appointment of Respondent  No.\n1, Shri Devi Lal as Deputy Prime Minister of India:\n   (i)  that

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [10]:
questions = [
    "why was the appeal rejected",
    # "How many hours of live sessions?",
    # "How many coding assignments are there in the program?",
    # "Is there a program certificate upon completion?",
    # "What programming language will be used in the program?",
    # "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: why was the appeal rejected
Answer: The question is not clear. Please provide more information and context to understand what is being asked. The passage provided does not include any context or explanation for the question. Therefore, it cannot be answered.

